In [3]:
import pandas as pd
from nltk.tokenize import WhitespaceTokenizer
import arabicstopwords.arabicstopwords as stp
import re
import qalsadi.lemmatizer as lemmatizer
from snowballstemmer import stemmer
from ar_corrector.corrector import Corrector

In [4]:
#-preprocessor
class Preprocessor():
    @staticmethod
    def process(sentence):
        # print("The Result is :")            
        s = Preprocessor.normDocument(sentence)
        s = Preprocessor.removestopwords(s)
        s = Preprocessor.stemDocument(s)
        # s = Preprocessor.lemmatizDocument(s)
        s = Preprocessor.correctDocument(s)
        return Preprocessor.tokenizeDocument(s)
    @staticmethod
    def tokenizeDocument(sentance):
        tokenizer = WhitespaceTokenizer()
        return tokenizer.tokenize(sentance)
    
    @staticmethod
    def removestopwords(sentence):
        terms=[]
        stopWords= set(stp.stopwords_list())
        for term in Preprocessor.tokenizeDocument(sentence) :
            if term not in stopWords :
                terms.append(term)
        terms = list(terms)
        return " ".join(terms)
    
    @staticmethod
    def lemmatizDocument(sentence):
        ar_lemmer = lemmatizer.Lemmatizer()
        return " ".join([ar_lemmer.lemmatize(i) for i in Preprocessor.tokenizeDocument(sentence)])     # type: ignore

    @staticmethod
    def stemDocument(sentence):
        ar_stemmer = stemmer("arabic")
        return " ".join([ar_stemmer.stemWord(i) for i in Preprocessor.tokenizeDocument(sentence)])    

    @staticmethod
    def normDocument(sentence):
        text = re.sub("[إأٱآا]", "ا", sentence)
        text = re.sub("ى", "ي", text)
        text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,،-./:;<=>؟?@[\]^_`{|}~"""), ' ', text)
        text = re.sub("ؤ", "ء", text)
        text = re.sub("ئ", "ء", text)
        text = re.sub("ة", "ه", text)
        return(text)
    
    @staticmethod
    def correctDocument(sentance):    
        corr = Corrector()        
        return " ".join([corr.contextual_correct(term) for term in Preprocessor.tokenizeDocument(sentance)])


In [5]:
d1 = "برنامج تعليمي للغة الإنجليزية و مسار سريع"
d2 = "تعلم الفهرسة الدلالية الكامنة"
d3 = "الكتاب في الفهرسة الدلالية"
d4 = "التقدم في البنية و الفهرسة الدلالية"
d5 = "تحليل تحليل البنية الكامنة"

def loadDocuments():
    return pd.DataFrame([
        [0,d1],
        [1,d2],
        [2,d3],
        [3,d4],
        [4,d5],
    ],columns=["id","text"])
df = loadDocuments()
df

,id,text
0,0,برنامج تعليمي للغة الإنجليزية و مسار سريع
1,1,تعلم الفهرسة الدلالية الكامنة
2,2,الكتاب في الفهرسة الدلالية
3,3,التقدم في البنية و الفهرسة الدلالية
4,4,تحليل تحليل البنية الكامنة


In [6]:
list_data = []
for i in range(df["id"].size):
   list_data.append(Preprocessor.process(df["text"][i]))

In [65]:
list_data

[['برنامج', 'تعليم', 'للغه', 'انجليزيه', 'مسار', 'سريع'],
 ['تعلم', 'فهرسه', 'دلالة', 'كامنه'],
 ['كتاب', 'فهرسه', 'دلالة'],
 ['تقدم', 'بنيه', 'فهرسه', 'دلالة'],
 ['تحليل', 'تحليل', 'بنيه', 'كامنه']]

In [7]:
dic = {0: ['برنامج', 'تعليم', 'للغه', 'انجليزيه', 'مسار', 'سريع'],
 1: ['تعلم', 'فهرسه', 'دلالة', 'كامنه'],
 2: ['كتاب', 'فهرسه', 'دلالة'],
 3: ['تقدم', 'بنيه', 'فهرسه', 'دلالة'],
 4: ['تحليل', 'تحليل', 'بنيه', 'كامنه']}
dic

{0: ['برنامج', 'تعليم', 'للغه', 'انجليزيه', 'مسار', 'سريع'],
 1: ['تعلم', 'فهرسه', 'دلالة', 'كامنه'],
 2: ['كتاب', 'فهرسه', 'دلالة'],
 3: ['تقدم', 'بنيه', 'فهرسه', 'دلالة'],
 4: ['تحليل', 'تحليل', 'بنيه', 'كامنه']}

In [8]:
df.insert(2,"ntext",list_data)

In [9]:
df

,id,text,ntext
0,0,برنامج تعليمي للغة الإنجليزية و مسار سريع,"[برنامج, تعليم, للغه, انجليزيه, مسار, سريع]"
1,1,تعلم الفهرسة الدلالية الكامنة,"[تعلم, فهرسه, دلالة, كامنه]"
2,2,الكتاب في الفهرسة الدلالية,"[كتاب, فهرسه, دلالة]"
3,3,التقدم في البنية و الفهرسة الدلالية,"[تقدم, بنيه, فهرسه, دلالة]"
4,4,تحليل تحليل البنية الكامنة,"[تحليل, تحليل, بنيه, كامنه]"


In [10]:
uniqterm = []
i = 0
for i in range(len(dic)):
    for terms in dic[i]:
        uniqterm.append(terms)
        i += 1
uniqterm = list(set(uniqterm))
uniqterm

['تعلم',
 'فهرسه',
 'بنيه',
 'تحليل',
 'تقدم',
 'مسار',
 'انجليزيه',
 'للغه',
 'سريع',
 'دلالة',
 'برنامج',
 'كامنه',
 'تعليم',
 'كتاب']

In [12]:
final_data ={}
termdoc = df.to_dict('list')
# for term1 in uniqterm:
#     temp = []
count = 0
for i, c in enumerate(termdoc['ntext']):
    # print(f"{i}    {c}")
    termList = []
    for uniq in uniqterm:
        for g in range(len(c)):
            if uniq in c:
                count += 1
        if count == 0:
            pass
        else:
            termList.append((i,count))
            final_data[uniq] = termList  
            count = 0  

final_data        

{'مسار': [(0, 6), (0, 6), (0, 6), (0, 6), (0, 6), (0, 6)],
 'انجليزيه': [(0, 6), (0, 6), (0, 6), (0, 6), (0, 6), (0, 6)],
 'للغه': [(0, 6), (0, 6), (0, 6), (0, 6), (0, 6), (0, 6)],
 'سريع': [(0, 6), (0, 6), (0, 6), (0, 6), (0, 6), (0, 6)],
 'برنامج': [(0, 6), (0, 6), (0, 6), (0, 6), (0, 6), (0, 6)],
 'تعليم': [(0, 6), (0, 6), (0, 6), (0, 6), (0, 6), (0, 6)],
 'تعلم': [(1, 4), (1, 4), (1, 4), (1, 4)],
 'فهرسه': [(3, 4), (3, 4), (3, 4), (3, 4)],
 'دلالة': [(3, 4), (3, 4), (3, 4), (3, 4)],
 'كامنه': [(4, 4), (4, 4), (4, 4)],
 'كتاب': [(2, 3), (2, 3), (2, 3)],
 'بنيه': [(4, 4), (4, 4), (4, 4)],
 'تقدم': [(3, 4), (3, 4), (3, 4), (3, 4)],
 'تحليل': [(4, 4), (4, 4), (4, 4)]}